# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"
# DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': 'c7eccbaa-eb5b-4185-bf06-b61e20c3b60b', 'created_at': '2025-08-18T02:49:08.897176+00:00', 'updated_at': '2025-08-18T02:49:08.897176+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
c7eccbaa-eb5b-4185-bf06-b61e20c3b60b


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [6]:
input = { "messages": "제주도 여행"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
제주도 여행을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. Introduction to Jeju Island - Introduce the unique aspects of Jeju island - its geography, culture, and climate. 
3. Must-see Attractions in Jeju - Explore the top tourist attractions and activities in Jeju, tailored for various preferences like nature, adventure, and culture. 
4. Local Cuisine and Dining - Guide to the best local dining experiences and must-try foods in Jeju, including both famous and hidden gem restaurants. 
5. Accommodation Options - Detailed comparison and advice on staying in Jeju, including hotels, guesthouses, and unique lodging experiences like hanok stays. 
6. Transport and Accessibility - Information on traveling within Jeju, covering public transport, car rentals, and other options for getting around the island. 
7. Seasonal Travel Tips - Best itineraries and travel tips tailored

In [7]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [8]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
제주도 여행을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. Introduction to Jeju Island - Introduce the unique aspects of Jeju island - its geography, culture, and climate. 
3. Must-see Attractions in Jeju - Explore the top tourist attractions and activities in Jeju, tailored for various preferences like nature, adventure, and culture. 
4. Local Cuisine and Dining - Guide to the best local dining experiences and must-try foods in Jeju, including both famous and hidden gem restaurants. 
5. Accommodation Options - Detailed comparison and advice on staying in Jeju, including hotels, guesthouses, and unique lodging experiences like hanok stays. 
6. Transport and Accessibility - Information on traveling within Jeju, covering public transport, car rentals, and other options for getting around the island. 
7. Seasonal Travel Tips - Best itineraries and travel tips tailored

In [9]:
len(state["values"]["slides"])

8

In [10]:
state["values"]["slides"]

[{'description': '제주도 여행에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'slide_type': 'title',
  'topic': '제주도 여행',
  'idx': 0,
  'name': '타이틀',
  'content': '## 제주도 여행 프레젠테이션 타이틀\n\n제주도는 한국의 자연과 문화가 아름답게 결합된 섬으로, 다양한 관광 명소와 활동을 제공합니다. "로맨틱 제주 길잡이"라는 타이틀은 제주도의 다채로운 풍경과 독특한 명소를 탐방하는 여행의 핵심을 잘 포착합니다[^1^]. 또한 이 타이틀은 제주도의 낭만적인 분위기를 강조하며, 관광객들에게 특별한 여행 경험을 제공하는 제주도의 매력을 잘 담아냅니다.\n\n### 자연과 문화의 축제\n\n제주도는 한라산을 포함한 다양한 자연경관과 독특한 문화적 역사로 유명합니다. 또한, 유네스코 세계자연유산으로 지정된 제주도의 지리적 특성과 관광지로서의 가치를 높여줍니다[^2^]. 이러한 배경에서 "자연과 문화의 축제"라는 부제목은 제주도를 방문하는 여행자들에게 그들이 경험하게 될 자연과 문화의 화합을 기대하게 합니다.\n\n### 제주도의 대표적 명소\n\n프레젠테이션의 주요 내용으로는 제주도의 대표적인 관광 명소를 예로 소개할 수 있습니다. 성산일출봉, 한라산 국립공원, 제주 올레길과 같은 자연 경관과 함께 제주 민속촌과 같은 역사적 명소들이 포함될 수 있습니다[^3^]. 이러한 장소들은 제주도 여행의 필수 방문지로, 각기 다른 테마와 분위기를 제공합니다.\n\n### 이색적인 체험과 활동\n\n제주도는 여행객들에게 이색적이고 다양한 체험 활동을 제공합니다. 제주 흙돼지나 감귤 농장에서의 체험, 바다가 보이는 멋진 카페에서의 휴식 등이 포함될 수 있습니다[^4^]. 이처럼 제주도는 단지 명소 관람에 그치지 않고, 지역의 특색있는 경험을 통해 더욱 깊이 있는 여행을 선사합니다.\n\n제주도

In [11]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])